<a href="https://colab.research.google.com/github/jumbokh/ML-Class/blob/main/notebooks/CH18_ChineseWords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Source: [tripadvisor.xlsx](https://drive.google.com/file/d/1kbXIX4QGvNMk7P5-mQD6kc5y6tbTsc8c/view?usp=sharing)
### 字典檔: [dict.txt.big](https://drive.google.com/file/d/1LY1Hjaq5GpAuWP_d7EgW5bGIGFZc6pKD/view?usp=sharing)
### 輔助檔: [stop.text](https://drive.google.com/file/d/1HIFZ17KgYEiBIz1drMG4eQoVtJdKVUR5/view?usp=sharing)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['DFKai-sb'] 
plt.rcParams['axes.unicode_minus'] = False
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')

df = pd.read_excel('tripadvisor.xlsx', parse_dates=['date'])
df.head()

,uid,rating,date,title,content
0,Kay C,4,2019-09-05,"還行, 回程延遲",位置空間還不錯。餐點也很可以。3-3機位。清潔度很不錯。對小朋友也還可以。出發的時間很準時。...
1,MinJer Lai,3,2019-09-05,空服員訓練仍有不足,"台北紐約航段有一個點心餐和兩個正餐, 點心餐就是堅果包和飲料\n在第一個正餐,我們被告知沒有..."
2,Rui,3,2019-09-04,舊機型沒個人娛樂、回程魚肉飯好吃,舊機型沒個人娛樂，只有抬頭電視可以看公放的電影、回程魚肉飯好吃。颱風剛過有小延誤，高的人坐起...
3,gigil169,4,2019-08-23,"準點, 對之前的猶豫已一掃而空","真的沒有讓人失望, 之前只坐過一次, 但還是會猶豫不決, 最後因為航班選擇比較多, 彈性大一..."
4,Wei-hsiang,4,2019-08-20,舒適,舒適平穩，並且提供餐點供乘客享用，座位上亦提供薄毯避免乘客受寒，座位前方有休閒娛樂系統，其中...


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   uid      2557 non-null   object        
 1   rating   2557 non-null   int64         
 2   date     2557 non-null   datetime64[ns]
 3   title    2557 non-null   object        
 4   content  2557 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 100.0+ KB


In [3]:
# ch18-3
size = df['rating'].value_counts().sort_index()
pct = df['rating'].value_counts(normalize=True).round(2).sort_index()
pd.DataFrame(zip(size, pct), columns=['次數', '百分比'], index=range(1,6))

,次數,百分比
1,52,0.02
2,72,0.03
3,287,0.11
4,1019,0.40
5,1127,0.44


In [4]:
# ch18-4
df.groupby(df['date'].dt.year)['rating'].agg(['size','mean'])

,size,mean
date,,
2016,810,4.251852
2017,795,4.270440
2018,685,4.144526
2019,267,4.082397


In [5]:
# ch18-5
import jieba
# 載入繁體字典
jieba.set_dictionary('dict.txt.big')
print(list(jieba.cut('下雨天留客天留我不留')))
# 將串列組合回字串，用空白做區隔
s = ' '.join(jieba.cut('下雨天留客天留我不留'))
print(s)

Building prefix dict from /content/dict.txt.big ...
Loading model from cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.452 seconds.
Prefix dict has been built successfully.


['下雨天', '留客', '天留', '我', '不留']
下雨天 留客 天留 我 不留


In [6]:
# ch18-6
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
bow = cv.fit_transform([s])
pd.DataFrame(bow.toarray(), columns=cv.get_feature_names())

,下雨天,不留,天留,留客
0,1,1,1,1


In [7]:
# ch18-7
cv = CountVectorizer(token_pattern='(?u)\\b\\w+\\b')
bow = cv.fit_transform([s])
pd.DataFrame(bow.toarray(), columns=cv.get_feature_names())

,下雨天,不留,天留,我,留客
0,1,1,1,1,1


In [8]:
df.loc[0, 'content']

'位置空間還不錯。餐點也很可以。3-3機位。清潔度很不錯。對小朋友也還可以。出發的時間很準時。但是回程就碰上延遲, 約40分鐘。沒有個人娛樂設施。'

In [9]:
# ch18-9
s = ' '.join(jieba.cut(df.iloc[0]['content']))
s

'位置 空間 還 不錯 。 餐點 也 很 可以 。 3 - 3 機位 。 清潔度 很 不錯 。 對 小朋友 也還 可以 。 出發 的 時間 很 準時 。 但是 回程 就 碰上 延遲 ,   約 40 分鐘 。 沒有 個人 娛樂 設施 。'

In [10]:
# ch18-10
# 從檔案讀入停用字，並做成串列
with open('stop.text','r', encoding='utf-8') as f:
    stops = f.read()
stops = stops.split('\n')

cv = CountVectorizer(token_pattern='(?u)\\b\\w+\\b', 
                     stop_words=stops)
bow = cv.fit_transform([s])
print(cv.get_feature_names())

['40', '不錯', '也還', '位置', '個人', '出發', '分鐘', '回程', '娛樂', '小朋友', '延遲', '時間', '機位', '沒有', '清潔度', '準時', '碰上', '空間', '約', '設施', '餐點']


In [11]:
# ch18-11
cv = CountVectorizer(token_pattern='(?u)\\b\\w+\\b', 
                     stop_words=stops, 
                     ngram_range=(1,2))
bow = cv.fit_transform([s])
print(cv.get_feature_names())

['40', '40 分鐘', '不錯', '不錯 小朋友', '不錯 餐點', '也還', '也還 出發', '位置', '位置 空間', '個人', '個人 娛樂', '出發', '出發 時間', '分鐘', '分鐘 沒有', '回程', '回程 碰上', '娛樂', '娛樂 設施', '小朋友', '小朋友 也還', '延遲', '延遲 約', '時間', '時間 準時', '機位', '機位 清潔度', '沒有', '沒有 個人', '清潔度', '清潔度 不錯', '準時', '準時 回程', '碰上', '碰上 延遲', '空間', '空間 不錯', '約', '約 40', '設施', '餐點', '餐點 機位']


In [12]:
# ch18-12
df['rating'] = (df['rating'] > 3).map({True:1 , False:0})
df['rating'].value_counts()

1    2146
0     411
Name: rating, dtype: int64

In [13]:
# ch18-13
X = df['content']
y = df['rating']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, random_state=42)

In [14]:
# ch18-14
def preprocessor(s):
    return ' '.join(jieba.cut(s))

print('斷字函數的結果：', preprocessor('下雨天留客天留我不留'))
cv = CountVectorizer(preprocessor=preprocessor,    
    token_pattern='(?u)\\b\\w+\\b', 
    stop_words=stops)
bow = cv.fit_transform(['下雨天留客天留我不留'])
pd.DataFrame(bow.toarray(), columns=cv.get_feature_names())

斷字函數的結果： 下雨天 留客 天留 我 不留


,下雨天,不留,天留,留客
0,1,1,1,1


In [15]:
# ch18-15
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

cv = CountVectorizer(preprocessor=preprocessor,    
    token_pattern='(?u)\\b\\w+\\b', 
    stop_words=stops)
model_pl = make_pipeline(cv, MultinomialNB())
model_pl.fit(X_train, y_train)
y_pred = model_pl.predict(X_test)
score = model_pl.score(X_test, y_test)
print('測試集的結果', score.round(3))
print(confusion_matrix(y_test, y_pred))
print('綜合報告')
print(classification_report(y_test, y_pred))

測試集的結果 0.844
[[ 14  73]
 [  7 418]]
綜合報告
              precision    recall  f1-score   support

           0       0.67      0.16      0.26        87
           1       0.85      0.98      0.91       425

    accuracy                           0.84       512
   macro avg       0.76      0.57      0.59       512
weighted avg       0.82      0.84      0.80       512

